In [1]:
import stata_setup
stata_setup.config("C:/Program Files/Stata17/", "mp")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706303466
  Licensed to: David Tomas Jacho-Chavez
               Emory University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


## Lasso IV

In [2]:
%%stata
use https://www.stata-press.com/data/r17/mroz, clear
vl create exogbase = (exper age husage kidslt6 kidsge6 city)
note: $exogbase initialized with 6 variables.
vl create instbase = (motheduc fatheduc huseduc)
note: $instbase initialized with 3 variables.


. use https://www.stata-press.com/data/r17/mroz, clear

. vl create exogbase = (exper age husage kidslt6 kidsge6 city)
note: $exogbase initialized with 6 variables.

. note: $exogbase initialized with 6 variables.

. vl create instbase = (motheduc fatheduc huseduc)
note: $instbase initialized with 3 variables.

. note: $instbase initialized with 3 variables.

. 


Let's add ```c.instbase#c.instbase``` to the list of potential instruments. The ```#``` is the factor-variable operator for interaction. It can interact categorical variables, continuous variables, or both. The ```c.``` prefix tells the ```#``` operator to treat the lists as continuous variables. ```#``` assumes categorical variables unless told otherwise.

In [3]:
%%stata
vl substitute inst = c.instbase c.instbase#c.instbase
macro list inst


. vl substitute inst = c.instbase c.instbase#c.instbase

. macro list inst
inst:           motheduc fatheduc huseduc c.motheduc#c.motheduc
                c.motheduc#c.fatheduc c.motheduc#c.huseduc
                c.fatheduc#c.fatheduc c.fatheduc#c.huseduc c.huseduc#c.huseduc

. 


We count nine terms—three linear terms and six interactions (including quadratic terms).

IV Lasso can be implemented in Stata using the downloadable package [ivlasso](https://statalasso.github.io/docs/pdslasso/). You would need to run the following commands on your local Stata installation while connected to the internet
```
ssc install ranktest
ssc install ivreg2
ssc install lassopack
net install pdslasso, ///
	from("https://raw.githubusercontent.com/statalasso/pdslasso/master/") 
```


In [4]:
%%stata
ivlasso lwage $exogbase (educ=$inst), robust

5.  (PDS/CHS) Selecting HD controls/IVs for endog regressor educ...
Selected: c.motheduc#c.huseduc c.fatheduc#c.huseduc c.huseduc#c.huseduc
7.  (CHS) Creating orthogonalized endogenous regressor educ...


Estimation results:

Specification:
Regularization method:                 lasso
Penalty loadings:                      heteroskedastic
Number of observations:                428
Exogenous (6):                         exper age husage kidslt6 kidsge6 city
Endogenous (1):                        educ
Unpenalized controls (1):              _cons
High-dim instruments (9):              motheduc fatheduc huseduc
                                       c.motheduc#c.motheduc
                                       c.motheduc#c.fatheduc
                                       c.motheduc#c.huseduc
                                       c.fatheduc#c.fatheduc
                                       c.fatheduc#c.huseduc c.huseduc#c.huseduc
Selected instruments (3):              c.motheduc#c.huseduc
  